In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential

import numpy as np
from glob import glob
import matplotlib.pyplot as plt

2023-07-21 12:33:50.335266: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-21 12:33:50.717904: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-21 12:33:51.495285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Проверим работу Cuda

In [2]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:GPU:0']


2023-07-21 12:33:54.899390: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-21 12:33:55.076891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-21 12:33:55.077234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
IMAGE_SIZE = [224, 224]

Проведем путь до файлов

In [4]:
train_loc = '/home/chris/Downloads/AI_course/NN_arch_and_Autocoder/Face_Mask_Dataset/Train'
test_loc = '/home/chris/Downloads/AI_course/NN_arch_and_Autocoder/Face_Mask_Dataset/Test'

Загрузим модель

In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50
resnet = ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=( 224 , 224  , 3) ,
    pooling='avg' ,
    classes=2
)
for layer in resnet.layers:
    layer.trainable = False

2023-07-21 12:33:59.279811: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-21 12:33:59.280207: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-21 12:33:59.280367: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [7]:
from glob import glob
folders = glob(train_loc + '/*')
folders
num_classes = len(folders)

In [8]:
x = Flatten()(resnet.output)

In [9]:
x = Dense(num_classes, activation = 'softmax')(x)

In [10]:
model = Model(inputs = resnet.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model.save_weights('orig.h5')

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [13]:
model.compile (
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [14]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)
training_set = train_datagen.flow_from_directory(
    train_loc,
    target_size = IMAGE_SIZE,
    batch_size = 32,
    class_mode = 'categorical' # As we have more than 2 so using categorical.. for 2 we might have used binary.
)
test_set = train_datagen.flow_from_directory(
    test_loc,
    target_size = IMAGE_SIZE,
    batch_size = 32,
    class_mode = 'categorical'
)

Found 10000 images belonging to 2 classes.
Found 993 images belonging to 2 classes.


In [15]:
import tensorflow
with tensorflow.device('/GPU:0'):
    
    history = model.fit_generator(
        training_set,
        validation_data = test_set,
        epochs = 6,
        steps_per_epoch = len(training_set),
        validation_steps = len(test_set)
    )

/tmp/ipykernel_2619/1042511051.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/6


2023-07-21 12:34:09.545135: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-21 12:34:12.610800: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-21 12:34:14.253146: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-21 12:34:15.594182: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x1f042400 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-21 12:34:15.594208: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-07-21 12:34:1

  1/313 [..............................] - ETA: 33:54 - loss: 0.9095 - accuracy: 0.5000

2023-07-21 12:34:16.025699: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 [==============================] - ETA: 0s - loss: 0.6562 - accuracy: 0.6066

2023-07-21 12:35:48.723637: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


313/313 [==============================] - 111s 334ms/step - loss: 0.6562 - accuracy: 0.6066 - val_loss: 0.6227 - val_accuracy: 0.6777
Epoch 2/6
313/313 [==============================] - 106s 339ms/step - loss: 0.6016 - accuracy: 0.6885 - val_loss: 0.5761 - val_accuracy: 0.7080
Epoch 3/6
313/313 [==============================] - 106s 339ms/step - loss: 0.5763 - accuracy: 0.7097 - val_loss: 0.5614 - val_accuracy: 0.6918
Epoch 4/6
313/313 [==============================] - 412s 1s/step - loss: 0.5485 - accuracy: 0.7462 - val_loss: 0.5352 - val_accuracy: 0.7936
Epoch 5/6
313/313 [==============================] - 551s 2s/step - loss: 0.5272 - accuracy: 0.7741 - val_loss: 0.5074 - val_accuracy: 0.8056
Epoch 6/6
313/313 [==============================] - 550s 2s/step - loss: 0.5106 - accuracy: 0.7845 - val_loss: 0.5005 - val_accuracy: 0.8207


In [16]:
model.save_weights('mask_reg.h5')

Вывод точности для датасета с масками

In [17]:
print ('Training Accuracy = ' + str(history.history['accuracy']))
print ('Validation Accuracy = ' + str(history.history['val_accuracy']))

Training Accuracy = [0.6065999865531921, 0.6884999871253967, 0.7096999883651733, 0.7462000250816345, 0.7741000056266785, 0.784500002861023]
Validation Accuracy = [0.6777442097663879, 0.7079557180404663, 0.6918429136276245, 0.7935549020767212, 0.8056395053863525, 0.8207452297210693]


In [18]:
model.load_weights('orig.h5')

In [19]:
train_loc = '/home/chris/Downloads/AI_course/NN_arch_and_Autocoder/weather_database/train'
test_loc = '/home/chris/Downloads/AI_course/NN_arch_and_Autocoder/weather_database//test'

In [20]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)
training_set = train_datagen.flow_from_directory(
    train_loc,
    target_size = IMAGE_SIZE,
    batch_size = 32,
    class_mode = 'categorical' # As we have more than 2 so using categorical.. for 2 we might have used binary.
)
test_set = train_datagen.flow_from_directory(
    test_loc,
    target_size = IMAGE_SIZE,
    batch_size = 32,
    class_mode = 'categorical'
)

Found 2067 images belonging to 2 classes.
Found 1724 images belonging to 2 classes.


In [21]:
with tensorflow.device('/GPU:0'):
    
    history = model.fit_generator(
        training_set,
        validation_data = test_set,
        epochs = 6,
        steps_per_epoch = len(training_set),
        validation_steps = len(test_set)
    )

/tmp/ipykernel_2619/925329771.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/6


2023-07-21 13:05:38.463232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


65/65 [==============================] - ETA: 0s - loss: 0.6780 - accuracy: 0.5854

2023-07-21 13:07:52.904752: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


65/65 [==============================] - 249s 4s/step - loss: 0.6780 - accuracy: 0.5854 - val_loss: 0.6418 - val_accuracy: 0.6531
Epoch 2/6
65/65 [==============================] - 236s 4s/step - loss: 0.6525 - accuracy: 0.6270 - val_loss: 0.6486 - val_accuracy: 0.6137
Epoch 3/6
65/65 [==============================] - 234s 4s/step - loss: 0.6431 - accuracy: 0.6183 - val_loss: 0.6226 - val_accuracy: 0.6595
Epoch 4/6
65/65 [==============================] - 235s 4s/step - loss: 0.6543 - accuracy: 0.6207 - val_loss: 0.6168 - val_accuracy: 0.6688
Epoch 5/6
65/65 [==============================] - 237s 4s/step - loss: 0.6290 - accuracy: 0.6512 - val_loss: 0.6149 - val_accuracy: 0.6734
Epoch 6/6
65/65 [==============================] - 234s 4s/step - loss: 0.6328 - accuracy: 0.6439 - val_loss: 0.6113 - val_accuracy: 0.6763


Вывод точности для датасета с погодой

In [ ]:
print ('Training Accuracy = ' + str(history.history['accuracy']))
print ('Validation Accuracy = ' + str(history.history['val_accuracy']))